# Datawrapper - making the charts with USGS data

- chart of population exposed by shake intensity
- chart of aftershocks

In [52]:
import pandas as pd
from datawrapper import Datawrapper
import os
import geopandas as gpd




In [51]:
# Récupère ta clé API depuis les variables d'environnement
api_key = os.getenv("DATAWRAPPER_API_KEY")  # remplace par le nom exact de ta variable

# Initialise Datawrapper
dw = Datawrapper(api_key)



# Tu peux maintenant utiliser dw.account_info()
info = dw.account_info()
print(info)

This method is deprecated and will be removed in a future version. Use get_account_info instead.


{'id': 489654, 'email': 'olaf.koenig@tamedia.ch', 'name': 'Olaf König', 'role': 'editor', 'language': 'en-US', 'presenceColor': 'color-dw-presence-30', 'avatar': None, 'customAvatar': None, 'teams': [{'id': 'tagesanzeiger', 'name': 'Tamedia', 'url': '/v3/teams/tagesanzeiger', 'active': False}], 'chartCount': 672, 'url': '/v3/users/489654', 'activeTeam': None, 'entitlements': {}}


## Import data

In [8]:
output_dir = "data_input/USGS"

In [49]:
### Data population exposure

In [27]:
df_exposure = pd.read_csv(f"{output_dir}/exposure_population_by_country_mmi.csv")
label_exposure = pd.read_csv(f"{output_dir}/mmi_intensity_category.csv")
print(df_exposure.head())
print(label_exposure.head())

df_exposure = pd.merge(
    df_exposure,
    label_exposure,
    how="left",
    left_on="mmi",
    right_on="Intensity"
)

df_exposure

  country  mmi  pop_exposure country_name_en country_name_fr country_name_de
0     ALL    1             0           Total           Total           Total
1     ALL    2             0           Total           Total           Total
2     ALL    3      32989235           Total           Total           Total
3     ALL    4     163708636           Total           Total           Total
4     ALL    5      24656409           Total           Total           Total
   Intensity Shaking_EN   Damage_EN     Shaking_DE    Damage_DE    Shaking_FR  \
0          1   Not felt         NaN  Nicht fühlbar        Keine  Non ressenti   
1          2       Weak         NaN        Schwach        Keine        Faible   
2          3       Weak         NaN        Schwach        Keine        Faible   
3          4      Light         NaN         Leicht        Keine         Léger   
4          5   Moderate  Very light         Mässig  Sehr gering        Modéré   

    Damage_FR  
0       Aucun  
1       Aucun  
2  

,country,mmi,pop_exposure,country_name_en,country_name_fr,country_name_de,Intensity,Shaking_EN,Damage_EN,Shaking_DE,Damage_DE,Shaking_FR,Damage_FR
0,ALL,1,0,Total,Total,Total,1,Not felt,NaN,Nicht fühlbar,Keine,Non ressenti,Aucun
1,ALL,2,0,Total,Total,Total,2,Weak,NaN,Schwach,Keine,Faible,Aucun
2,ALL,3,32989235,Total,Total,Total,3,Weak,NaN,Schwach,Keine,Faible,Aucun
3,ALL,4,163708636,Total,Total,Total,4,Light,NaN,Leicht,Keine,Léger,Aucun
4,ALL,5,24656409,Total,Total,Total,5,Moderate,Very light,Mässig,Sehr gering,Modéré,Très léger
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,TH,6,141478,Thailand,Thaïlande,Thailand,6,Strong,Light,Stark,Gering,Fort,Léger
96,TH,7,0,Thailand,Thaïlande,Thailand,7,Very strong,Moderate,Sehr stark,Mittel,Très fort,Modéré
97,TH,8,0,Thailand,Thaïlande,Thailand,8,Severe,Moderate/heavy,Heftig,Mittel/hoch,Sévère,Modéré/important
98,TH,9,0,Thailand,Thaïlande,Thailand,9,Violent,Heavy,Heftig,Hoch,Violent,Important


### Cleaning exposure and transform to wide

In [41]:
df_wide = (
    df_exposure[df_exposure["country"].isin(["ALL", "MM"])]
    [["mmi", "Shaking_EN", "country_name_en", "pop_exposure"]]
    .pivot(index=["mmi", "Shaking_EN"], columns="country_name_en", values="pop_exposure")
    .reset_index()
)
df_wide = df_wide[~df_wide['mmi'].isin([1, 2])]

df_wide["pct_myanmar"] = (
    df_wide["Myanmar (Burma)"] / df_wide["Total"] * 100
).clip(upper=100).round(1)


### Create dw exposure chart

In [31]:


folder_id = "332403"

In [ ]:
# # The function create_chart() can take four arguments: title, chart_type, data, and folder_id.


# dw_exposure = dw.create_chart(
#     title="Population exposure at the earthquake",
#     chart_type="tables",
#     data = df_wide,
#     folder_id=folder_id
# )

In [ ]:
# dw_exposure_id = dw_exposure['id']
# print(dw_exposure_id)

dw_exposure_id = "dQpEY"

dQpEY


#### Update data

In [43]:
dw.add_data(dw_exposure_id, df_wide)

True

#### Publish chart

In [47]:
dw.publish_chart(dw_exposure_id)

dw_exposure_iframe = dw.get_iframe_code(dw_exposure_id)

'<iframe title="The Earthquake’s Impact Fell Heaviest on Myanmar’s Population" aria-label="Table" id="datawrapper-chart-dQpEY" src="https://datawrapper.dwcdn.net/dQpEY/4/" scrolling="no" frameborder="0" style="border: none;" width="320" height="446" data-external="1"></iframe>'

### Aftershocks data

In [59]:
dw_aftershocks = gpd.read_file(f"{output_dir}/aftershock_with_distance.gpkg")
dw_aftershocks

,id,time,magnitude,depth,latitude,longitude,magtype,gap,rms,horizontal_error,vertical_error,distance_to_fault_m,distance_to_fault_km,geometry
0,us7000pn9s,2025-03-28 06:20:52.715000+00:00,7.7,10.0,22.0110,95.9363,None,None,None,None,None,4.841912e+03,4.841912,POINT (95.9363 22.011)
1,us7000pn9z,2025-03-28 06:32:04.777000+00:00,6.7,10.0,21.6975,95.9690,None,None,None,None,None,2.036882e+03,2.036882,POINT (95.969 21.6975)
2,us7000pncy,2025-03-28 06:39:14.645000+00:00,4.8,10.0,19.8979,95.8026,None,None,None,None,None,3.629403e+04,36.294033,POINT (95.8026 19.8979)
3,us7000pncv,2025-03-28 06:42:24.760000+00:00,4.9,10.0,21.8377,95.8747,None,None,None,None,None,1.157123e+04,11.571228,POINT (95.8747 21.8377)
4,us7000pnb6,2025-03-28 06:45:44.906000+00:00,4.9,10.0,19.1284,96.2075,None,None,None,None,None,6.474854e+03,6.474854,POINT (96.2075 19.1284)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,us6000q72i,2025-04-18 18:01:22.215000+00:00,4.1,10.0,20.4354,96.1272,None,None,None,None,None,4.795000e+03,4.795000,POINT (96.1272 20.4354)
70,us6000q7gc,2025-04-21 00:03:23.936000+00:00,4.2,10.0,29.1753,86.9720,None,None,None,None,None,1.168997e+06,1168.996823,POINT (86.972 29.1753)
71,us6000q7ge,2025-04-21 01:07:50.979000+00:00,4.7,10.0,32.5723,93.3067,None,None,None,None,None,1.149769e+06,1149.768632,POINT (93.3067 32.5723)
72,us7000puj0,2025-04-23 16:09:01.885000+00:00,4.9,10.0,26.4164,100.0261,None,None,None,None,None,6.011740e+05,601.174035,POINT (100.0261 26.4164)


### Filter distance to fault

In [81]:

# Filtering distance
distance_to_fault = 100



dw_aftershocks_filtered = dw_aftershocks[dw_aftershocks["distance_to_fault_km"] <= distance_to_fault]


### Clean time and accentuate sizes differences

In [82]:

df_aftershocks_filtered = dw_aftershocks_filtered.drop(columns="geometry").copy()

# Conversion en datetime, si nécessaire
df_aftershocks_filtered["time"] = pd.to_datetime(df_aftershocks_filtered["time"])

# Chronologie fine pour DW
df_aftershocks_filtered["datetime"] = df_aftershocks_filtered["time"].dt.strftime('%Y-%m-%d %H:%M')
df_aftershocks_filtered["iso8601"] = df_aftershocks_filtered["time"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

# Ajout du radius avec une fonction quadratique
def mag_to_radius(mag, base=2):
    return base + (mag - 3.5)**2.5

df_aftershocks_filtered["radius"] = df_aftershocks_filtered["magnitude"].apply(mag_to_radius)

# Sélection des colonnes clés
cols = ["id", "magnitude", "radius", "distance_to_fault_km", "datetime", "iso8601"]
df_aftershock_for_dw = df_aftershocks_filtered[cols].copy()



### Create chart for aftershocks

In [ ]:
# dw_aftershock = dw.create_chart(
#     title="Aftershocks",
#     chart_type="tables",
#     data = df_aftershock_for_dw,
#     folder_id=folder_id
# )

In [ ]:
# dw_aftershock_id = dw_aftershock['id']
# print(dw_aftershock_id)

dw_aftershock_id = "UWXHh"

UWXHh


In [83]:
# Update chart

dw.add_data(dw_aftershock_id, df_aftershock_for_dw)

True

In [73]:
dw_aftershocks_filtered

,id,time,magnitude,depth,latitude,longitude,magtype,gap,rms,horizontal_error,vertical_error,distance_to_fault_m,distance_to_fault_km,geometry
0,us7000pn9s,2025-03-28 06:20:52.715000+00:00,7.7,10.000,22.0110,95.9363,None,None,None,None,None,4841.911795,4.841912,POINT (95.9363 22.011)
1,us7000pn9z,2025-03-28 06:32:04.777000+00:00,6.7,10.000,21.6975,95.9690,None,None,None,None,None,2036.881791,2.036882,POINT (95.969 21.6975)
2,us7000pncy,2025-03-28 06:39:14.645000+00:00,4.8,10.000,19.8979,95.8026,None,None,None,None,None,36294.032829,36.294033,POINT (95.8026 19.8979)
3,us7000pncv,2025-03-28 06:42:24.760000+00:00,4.9,10.000,21.8377,95.8747,None,None,None,None,None,11571.227728,11.571228,POINT (95.8747 21.8377)
4,us7000pnb6,2025-03-28 06:45:44.906000+00:00,4.9,10.000,19.1284,96.2075,None,None,None,None,None,6474.853617,6.474854,POINT (96.2075 19.1284)
5,us6000q4ji,2025-03-28 06:57:53.711000+00:00,4.5,10.000,22.0976,96.0667,None,None,None,None,None,9169.993542,9.169994,POINT (96.0667 22.0976)
6,us6000q4jt,2025-03-28 07:01:28.860000+00:00,4.4,10.000,22.0534,95.8669,None,None,None,None,None,11719.597959,11.719598,POINT (95.8669 22.0534)
7,us7000pnb8,2025-03-28 07:27:47.907000+00:00,4.6,10.000,22.7436,95.8927,None,None,None,None,None,31285.612831,31.285613,POINT (95.8927 22.7436)
8,us7000pnf1,2025-03-28 07:33:37.622000+00:00,4.4,10.000,22.6287,95.7353,None,None,None,None,None,30302.311849,30.302312,POINT (95.7353 22.6287)
9,us7000pnba,2025-03-28 07:36:58.799000+00:00,4.6,10.000,22.7673,95.8531,None,None,None,None,None,35080.863305,35.080863,POINT (95.8531 22.7673)


In [84]:
# S'assurer que le DataFrame est trié par 'time' croissante
df_aftershocks_filtered = df_aftershocks_filtered.sort_values("time")

# Prendre la date du tout premier aftershock comme point de départ
first_time = df_aftershocks_filtered["time"].iloc[0]

# Définir la fenêtre de 6 jours après ce timestamp
window_end = first_time + pd.Timedelta(days=6)

# Filtrer les aftershocks dans cette fenêtre
after_6days = df_aftershocks_filtered[
    (df_aftershocks_filtered["time"] >= first_time) &
    (df_aftershocks_filtered["time"] < window_end)
]

count = len(after_6days)
print(f"Nombre de tremblements de terre dans les 6 jours à partir du premier : {count}")


Nombre de tremblements de terre dans les 6 jours à partir du premier : 38
